## Step_1: Loading the dataset

In [ ]:
!wget -q https://downloads.cs.stanford.edu/nlp/data/colbert/colbertv2/lotte.tar.gz
!tar -xzf lotte.tar.gz
!sudo chown -R daemon:daemon lotte/

In [ ]:
!pip install lotte -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 22.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.7/76.7 KB 7.8 MB/s eta 0:00:00


In [ ]:
!pip freeze | grep lotte

In [ ]:
import urllib.request 
from bs4 import BeautifulSoup 
import re
import time

# let's import ES
# from elasticsearch import Elasticsearch
from lotte import lifestyle

In [ ]:
%%bash --bg
sudo -H -u daemon lotte/bin/lotte

%%bash
ps -ef | grep lotte

In [ ]:
!pip install ir_datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.5/311.5 KB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 KB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 69.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.5/112.5 KB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.3/293.3 KB 32.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for warc3-wet-clueweb09: filename=warc3_wet_clueweb09-0.2.5-py3-none-any.whl size=18918 sha256=973dadc5e40f02ae459da6b41a96cfacdec965e6ecdc907e686c54a59decb510
  Stored in directory: /root/.cache/pip/wheels/7f/22/ed/a11944d7fdf4e94c4206a3f760d385122a4d34d8acc12f71a3
  Created wheel for cbor: filename=cbor-1.0.0-cp39-cp39-linux_x86_64.whl size=57301 sha256=428d8568f8c4f60b92e25a4e9af75b06b9988207c35

In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 40.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 17.7 MB/s eta 0:00:00


In [ ]:
import json
import codecs
from typing import NamedTuple, Dict, List
import ir_datasets
from ir_datasets.util import TarExtractAll, Cache, RelativePath, Lazy, Migrator
from ir_datasets.datasets.base import Dataset, YamlDocumentation, FilteredQueries
from ir_datasets.formats import TsvDocs, TsvQueries, BaseQrels, GenericDoc, GenericQuery, TrecQrel
from ir_datasets.indices import PickleLz4FullStore

_logger = ir_datasets.log.easy()


NAME = 'lotte'
QRELS_DEFS = {1: 'Answer upvoted or accepted on stack exchange'}



class LotteQrels(BaseQrels):
    def __init__(self, qrels_dlc):
        self._qrels_dlc = qrels_dlc

    def qrels_path(self):
        return self._qrels_dlc.path()

    def qrels_iter(self):
        with self._qrels_dlc.stream() as f:
            for line in f:
                data = json.loads(line)
                for did in data['answer_pids']:
                    yield TrecQrel(str(data['qid']), str(did), 1, "0")

    def qrels_cls(self):
        return TrecQrel

    def qrels_defs(self):
        return QRELS_DEFS


def _init():
    base_path = ir_datasets.util.home_path()/NAME
    dlc = ir_datasets.util.DownloadConfig.context(NAME, base_path)
    documentation = YamlDocumentation(f'docs/{NAME}.yaml')

    base_dlc = TarExtractAll(dlc['source'], base_path/'lotte_extracted')

    base = Dataset(documentation('_'))

    subsets = {}

    domains = [
        ('lifestyle',),
        ('recreation',),
        ('science',),
        ('technology',),
        ('writing',),
        ('pooled',),
    ]

    for (domain,) in domains:
        for split in ['dev', 'test']:
            corpus = TsvDocs(RelativePath(base_dlc, f'lotte/{domain}/{split}/collection.tsv'), lang='en')
            subsets[f'{domain}/{split}'] = Dataset(
                corpus,
                documentation(f'{domain}/{split}')
            )
            for qtype in ['search', 'forum']:
                subsets[f'{domain}/{split}/{qtype}'] = Dataset(
                    corpus,
                    TsvQueries(RelativePath(base_dlc, f'lotte/{domain}/{split}/questions.{qtype}.tsv'), lang='en'),
                    LotteQrels(RelativePath(base_dlc, f'lotte/{domain}/{split}/qas.{qtype}.jsonl')),
                    documentation(f'{domain}/{split}/{qtype}')
                )

    # ir_datasets.registry.register(NAME, base)
    # for s in sorted(subsets):
    #     ir_datasets.registry.register(f'{NAME}/{s}', subsets[s])

    return base, subsets


base, subsets = _init()

In [ ]:
# Copyright 2020 The HuggingFace Datasets Authors and the current dataset script contributor.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# TODO: Address all TODOs and remove all explanatory comments
"""TODO: Add a description here."""


import csv
import json
import os

import datasets
import pandas as pd
import zipfile


# TODO: Add BibTeX citation
# Find for instance the citation on arxiv or on the dataset repo/website
_CITATION = """\
@inproceedings{santhanam-etal-2022-colbertv2,
    title = "{C}ol{BERT}v2: Effective and Efficient Retrieval via Lightweight Late Interaction",
    author = "Santhanam, Keshav  and
      Khattab, Omar  and
      Saad-Falcon, Jon  and
      Potts, Christopher  and
      Zaharia, Matei",
    booktitle = "Proceedings of the 2022 Conference of the North American Chapter of the Association for Computational Linguistics: Human Language Technologies",
    month = jul,
    year = "2022",
    address = "Seattle, United States",
    publisher = "Association for Computational Linguistics",
    url = "https://aclanthology.org/2022.naacl-main.272",
    pages = "3715--3734",
    abstract = "Neural information retrieval (IR) has greatly advanced search and other knowledge-intensive language tasks. While many neural IR methods encode queries and documents into single-vector representations, late interaction models produce multi-vector representations at the granularity of each token and decompose relevance modeling into scalable token-level computations. This decomposition has been shown to make late interaction more effective, but it inflates the space footprint of these models by an order of magnitude. In this work, we introduce Maize, a retriever that couples an aggressive residual compression mechanism with a denoised supervision strategy to simultaneously improve the quality and space footprint of late interaction. We evaluate Maize across a wide range of benchmarks, establishing state-of-the-art quality within and outside the training domain while reducing the space footprint of late interaction models by 6{--}10x.",
}
"""

# TODO: Add description of the dataset here
# You can copy an official description
_DESCRIPTION = """\
LoTTE Passages Dataset for ColBERTv2
"""

# TODO: Add a link to an official homepage for the dataset here
_HOMEPAGE = "https://huggingface.co/datasets/colbertv2/lotte"

# TODO: Add the licence for the dataset here if you can find it
_LICENSE = ""

_URL = {
            "pooled": "https://huggingface.co/datasets/colbertv2/lotte/resolve/main/pooled/",
            "lifestyle": "https://huggingface.co/datasets/colbertv2/lotte/resolve/main/lifestyle/",
            "recreation": "https://huggingface.co/datasets/colbertv2/lotte/resolve/main/recreation/",
            "science": "https://huggingface.co/datasets/colbertv2/lotte/resolve/main/science/",
            "technology": "https://huggingface.co/datasets/colbertv2/lotte/resolve/main/technology/",
            "writing": "https://huggingface.co/datasets/colbertv2/lotte/resolve/main/writing/"
       }




# TODO: Name of the dataset usually match the script name with CamelCase instead of snake_case
class NewDataset(datasets.GeneratorBasedBuilder):
    """TODO: Short description of my dataset."""

    VERSION = datasets.Version("1.1.0")

    # This is an example of a dataset with multiple configurations.
    # If you don't want/need to define several sub-sets in your dataset,
    # just remove the BUILDER_CONFIG_CLASS and the BUILDER_CONFIGS attributes.

    # If you need to make complex sub-parts in the datasets with configurable options
    # You can create your own builder configuration class to store attribute, inheriting from datasets.BuilderConfig
    # BUILDER_CONFIG_CLASS = MyBuilderConfig

    # You will be able to load one or the other configurations in the following list with
    # data = datasets.load_dataset('my_dataset', 'first_domain')
    # data = datasets.load_dataset('my_dataset', 'second_domain')
    BUILDER_CONFIGS = [
        datasets.BuilderConfig(name="pooled", version=VERSION, description=""),
        datasets.BuilderConfig(name="lifestyle", version=VERSION, description=""),
        datasets.BuilderConfig(name="recreation", version=VERSION, description=""),
        datasets.BuilderConfig(name="science", version=VERSION, description=""),
        datasets.BuilderConfig(name="technology", version=VERSION, description=""),
        datasets.BuilderConfig(name="writing", version=VERSION, description=""),
        #datasets.BuilderConfig(name="pooled_search_valid", version=VERSION, description=""),
        #datasets.BuilderConfig(name="pooled_search_test", version=VERSION, description=""),
        #datasets.BuilderConfig(name="pooled_forum_valid", version=VERSION, description=""),
        #datasets.BuilderConfig(name="pooled_forum_test", version=VERSION, description=""),
        #datasets.BuilderConfig(name="lifestyle_search", version=VERSION, description=""),
        #datasets.BuilderConfig(name="lifestyle_forum", version=VERSION, description=""),
        #datasets.BuilderConfig(name="recreation_search", version=VERSION, description=""),
        #datasets.BuilderConfig(name="recreation_forum", version=VERSION, description=""),
        #datasets.BuilderConfig(name="science_search", version=VERSION, description=""),
        #datasets.BuilderConfig(name="science_forum", version=VERSION, description=""),
        #datasets.BuilderConfig(name="technology_search", version=VERSION, description=""),
        #datasets.BuilderConfig(name="technology_forum", version=VERSION, description=""),
        #datasets.BuilderConfig(name="writing_search", version=VERSION, description=""),
        #datasets.BuilderConfig(name="writing_forum", version=VERSION, description=""),
    ]

    DEFAULT_CONFIG_NAME = "pooled"  # It's not mandatory to have a default configuration. Just use one if it make sense.

    def _info(self):
        # TODO: This method specifies the datasets.DatasetInfo object which contains informations and typings for the dataset
        features = datasets.Features(
            {
                "qid": datasets.Value("int32"),
                "query": datasets.Value("string"),
                "author": datasets.Value("string"),
                "answers": datasets.Sequence(feature={"views": datasets.Value(dtype='int32', id=None),
                                                      "score": datasets.Value(dtype='int32', id=None),
                                                      "answer_pids": datasets.Value(dtype='int32', id=None)})
            }
        )
        return datasets.DatasetInfo(
            # This is the description that will appear on the datasets page.
            description=_DESCRIPTION,
            # This defines the different columns of the dataset and their types
            features=features,  # Here we define them above because they are different between the two configurations
            # If there's a common (input, target) tuple from the features, uncomment supervised_keys line below and
            # specify them. They'll be used if as_supervised=True in builder.as_dataset.
            # supervised_keys=("sentence", "label"),
            # Homepage of the dataset for documentation
            homepage=_HOMEPAGE,
            # License for the dataset if available
            license=_LICENSE,
            # Citation for the dataset
            citation=_CITATION,
        )

    def _split_generators(self, dl_manager):
        # TODO: This method is tasked with downloading/extracting the data and defining the splits depending on the configuration
        # If several configurations are possible (listed in BUILDER_CONFIGS), the configuration selected by the user is in self.config.name

        # dl_manager is a datasets.download.DownloadManager that can be used to download and extract URLS
        # It can accept any type or nested list/dict and will give back the same structure with the url replaced with path to local files.
        # By default the archives will be extracted and a path to a cached folder where they are extracted is returned instead of the archive
        
        _URLS = {
            "forum_dev": _URL[self.config.name] + "dev_qas.forum.jsonl",
            "forum_test": _URL[self.config.name] + "test_qas.forum.jsonl",
            "search_dev": _URL[self.config.name] + "dev_qas.search.jsonl",
            "search_test": _URL[self.config.name] + "test_qas.search.jsonl",
        }

        downloaded_files = dl_manager.download_and_extract(_URLS)

        return [
            datasets.SplitGenerator(name="forum_dev", gen_kwargs={"filepath": downloaded_files["forum_dev"]}),
            datasets.SplitGenerator(name="forum_test", gen_kwargs={"filepath": downloaded_files["forum_test"]}),
            datasets.SplitGenerator(name="search_dev", gen_kwargs={"filepath": downloaded_files["search_dev"]}),
            datasets.SplitGenerator(name="search_test", gen_kwargs={"filepath": downloaded_files["search_test"]}),
        ]

    # method parameters are unpacked from `gen_kwargs` as given in `_split_generators`
    def _generate_examples(self, filepath):
        # TODO: This method handles input defined in _split_generators to yield (key, example) tuples from the dataset.
        # The `key` is for legacy reasons (tfds) and is not important in itself, but must be unique for each example.

        print("Generating examples")

        with open(filepath, encoding="utf-8") as f:

            for key, row in enumerate(f):
                
                data = json.loads(row)

                answers = {}

                if "score" in data.keys():
                    answers.update({"score": [data['score']]})
                else:
                    answers.update({"score": []})

                if "views" in data.keys():
                    answers.update({"views": [data['views']]})
                else:
                    answers.update({"views": []})

                answers.update({"answer_pids": data['answer_pids']})

                if "question_author" in data.keys():
                    author = data['question_author']
                else:
                    author = ""


                yield key, {
                        "qid": data["qid"],
                        "query": data["query"],
                        "author": author,
                        "answers": answers
                }

In [ ]:
datasets

<module 'datasets' from '/usr/local/lib/python3.9/dist-packages/datasets/__init__.py'>

In [ ]:
import ir_datasets
dataset_doc = ir_datasets.load('lotte/lifestyle/dev')
# Documents
# GenericDoc(doc_id='0', text='The presence of communication amid scientific minds was equa...
# GenericDoc(doc_id='1', text='The Manhattan Project and its atomic bomb helped bring an en...
# ...

In [ ]:
import ir_datasets
dataset_query = ir_datasets.load("lotte/lifestyle/dev/search")
# namedtuple<query_id, text>

## Step_2: Using TFIDF to retrieve top 5 documents

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import nltk
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
docs = [doc.text for doc in dataset_doc.docs_iter()]
doc_ids = [doc.doc_id for doc in dataset_doc.docs_iter()]
queries = [query.text for query in dataset_query.queries_iter()]
query_ids = [query.query_id for query in dataset_query.queries_iter()]

In [ ]:
# create the tf-idf vectorizer
vectorizer = TfidfVectorizer()

# fit the vectorizer on the documents
tfidf_matrix_data = vectorizer.fit_transform(docs)

In [ ]:
# define a function to retrieve relevant documents using the query
def retrieve_documents(query, n=5):
    # transform the doc and query into a tf-idf vector
    query_vec = vectorizer.transform([query])

    # compute the cosine similarity between the query vector and all the document vectors
    # scores = tfidf_matrix_data.dot(query_vec.T).toarray().squeeze()

    # sort the document IDs based on their scores
    # sorted_doc_ids = [doc_id for _, doc_id in sorted(zip(scores, doc_ids), reverse=True)]

    # return the top-n relevant documents
    # return sorted_doc_ids[:n]

    cosine_similarities = cosine_similarity(tfidf_matrix_data, query_vec).flatten()

    sorted_doc_ids = [doc_id for _, doc_id in sorted(zip(cosine_similarities, doc_ids), reverse=True)]

    return sorted_doc_ids[:n]

In [ ]:
id = query_ids[0]  # 0th id
query = queries[int(id)]

print(query)

how much should i feed my 1 year old english mastiff?


In [ ]:
retrieve_list = retrieve_documents(query)  # Will return top 5 documents having the query 
print(retrieve_list)

['4619', '42913', '54068', '55175', '49783']


In [ ]:
for i in (retrieve_list):
  print(i,":", docs[int(i)])

4619 : I have a 2 1/2 year old bull mastiff. I have been feeding him Blue Buffalo since I got him at 8 weeks old. He is very lean and active for a bull mastiff. I feed him about 3-4 cups twice a day which averages about 130.00 a month. It is very important that you can afford this breed. I just had to take mine to the vet because he developed some sort of allergies on his skin, eyes and ears and the vet bill was $210.00 with all his medication. This wasn't an option I had to take him an get all his meds or he would have gotten worse. They're just like your children, you can expect things to come up and you need to be able to care for them.
42913 : I think the key is to provide as much language exposure as possible, but in a natural way. Natural sources could be enrolling the child in an English-speaking Kindergarten, TV shows in English, frequently having English-speaking visitors or babysitters, etc. If you don't have any natural sources of English, then it's up to you alone. I'm face